In [ ]:
#Set up BigQuery client
from google.cloud import bigquery

project_id = "oamk-476515"
client = bigquery.Client(project=project_id)

c:\Users\nicot\code\data-engineering\.venv\Lib\site-packages\google\auth\_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
query = """
CREATE OR REPLACE TABLE `oamk-476515.G4_daily_user.G4_daily_user_data` AS
SELECT
  user_pseudo_id,
  COUNT(event_name) AS event_count,
  SUM(ecommerce.purchase_revenue_in_usd) AS total_revenue
FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20210115' AND '20210131'
GROUP BY
  user_pseudo_id
ORDER BY
  total_revenue DESC;
"""

job = client.query(query)
job.result()

print("Query completed (table created in BigQuery)")

Query completed (table created in BigQuery)


In [ ]:
from google.cloud import bigquery

# Initialize client
project_id = "oamk-476515"
client = bigquery.Client(project=project_id)

# Dataset and table destination
dataset_id = f"{project_id}.G4_daily_user"
table_id = f"{dataset_id}.G4_daily_user_data"

# SQL query
query = """
SELECT
  user_pseudo_id,
  COUNT(event_name) AS event_count,
  SUM(
    CAST(
      (SELECT ep.value.float_value 
       FROM UNNEST(event_params) ep 
       WHERE ep.key = "purchase_revenue_in_usd"
      ) AS FLOAT64
    )
  ) AS total_revenue
FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20210115' AND '20210131'
GROUP BY
  user_pseudo_id
ORDER BY
  total_revenue DESC
"""

# Configure destination
job_config = bigquery.QueryJobConfig(
    destination=table_id,
    write_disposition="WRITE_TRUNCATE"
)

# Run query directly in BigQuery
query_job = client.query(query, job_config=job_config)
query_job.result()  # wait for completion

print(f"Query executed successfully (results saved to {table_id})")


c:\Users\nicot\code\data-engineering\.venv\Lib\site-packages\google\auth\_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


BadRequest: 400 Cannot set destination table in jobs with DDL statements; reason: invalidQuery, message: Cannot set destination table in jobs with DDL statements

Location: US
Job ID: 5b2e18ce-9aa6-4ad0-890c-bf65907282d5
